In [ ]:
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

In [ ]:
# Load the MNIST dataset
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [ ]:
# Normalise pixel values to be between 0 and 1
X = X / 255.0

In [ ]:
# Convert labels to one hot encoding
num_classes = 10
y = np.eye(num_classes)[y.astype(int)]

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the sigmoid activation function
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [ ]:
# Define the sigmoid derivative
def sigmoid_derivative(x):
  return sigmoid(x) * (1 - sigmoid(x))

In [ ]:
# Define the neural network architecture
class NeuralNetwork:
  def __init__(self, input_size, hidden_size, output_size):
    # Initialize weights with random values
    self.weights1 = np.random.randn(input_size, hidden_size)
    self.weights2 = np.random.randn(hidden_size, output_size)
    self.bias1 = np.zeros((1, hidden_size))
    self.bias2 = np.zeros((1, output_size))

  def feedforward(self, X):
    # Calculate the activations of the hidden layer
    hidden_layer_activation = np.dot(X, self.weights1) + self.bias1
    hidden_layer_output = sigmoid(hidden_layer_activation)

    # Calculate the output layer activation
    output_layer_activation = np.dot(hidden_layer_output, self.weights2) + self.bias2
    output_layer_output = sigmoid(output_layer_activation)

    return hidden_layer_output, output_layer_output

  def backpropagation(self, X, y, hidden_layer_output, output_layer_output, learning_rate):
    # Calculate the error at the output layer
    output_error = output_layer_output - y

    # Calculate the slope of the output layer
    output_delta = output_error * sigmoid_derivative(output_layer_output)

    # Calculate the error at the hidden layer
    hidden_layer_error = output_delta.dot(self.weights2.T)

    # Calculate the slope of the hidden layer
    hidden_layer_delta = hidden_layer_error * sigmoid_derivative(hidden_layer_output)

    # Update weights and biases
    self.weights2 -= learning_rate * hidden_layer_output.T.dot(output_delta)
    self.bias2 -= learning_rate * np.sum(output_delta, axis=0, keepdims=True)
    self.weights1 -= learning_rate * X.T.dot(hidden_layer_delta)
    self.bias1 -= learning_rate * np.sum(hidden_layer_delta, axis=0, keepdims=True)

  def train(self, X_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            for i in range(len(X_train)):
                # Forward pass
                hidden_layer_output, output_layer_output = self.feedforward(X_train[i:i+1])

                # Backward pass
                self.backpropagation(X_train[i:i+1], y_train[i:i+1], hidden_layer_output, output_layer_output, learning_rate)

            # Print progress
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}")

  def predict(self, X_test):
      _, output_layer_output = self.feedforward(X_test)
      return np.argmax(output_layer_output, axis=1)

In [ ]:
# Create an instance of the neural network
input_size = 784
hidden_size = 128
output_size = 10
nn = NeuralNetwork(input_size, hidden_size, output_size)

In [ ]:
# Train the neural network
nn.train(X_train, y_train, epochs=100, learning_rate=0.1)

In [ ]:
# Make predictions on the test set
y_pred = nn.predict(X_test)

In [ ]:
score = accuracy_score(np.argmax(y_test, axis=1), y_pred)
print(f"Accuracy: {score}")

In [ ]:
# Visualize some predictions
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap='gray')
    plt.title(f"Predicted: {y_pred[i]}")
    plt.axis('off')
plt.show()